Declare valid hindi literals

In [1]:
a = list(set(list("ऀँंःऄअआइईउऊऋऌऍऎएऐऑऒओऔकखगघङचछजझञटठडढणतथदधनऩपफबभमयरऱलळऴवशषसहऺऻ़ऽािीुूृॄॅॆेैॉॊोौ्ॎॏॐ॒॑॓॔ॕॖॗक़ख़ग़ज़ड़ढ़फ़य़ॠॡॢॣ।॥॰ॱॲॳॴॵॶॷॸॹॺॻॼॽॾॿ-")))


Parse Hindimonocop05.export file and create a list

In [196]:
from tqdm import tqdm_notebook

genderList = set()
ind = 0
with open('hindmonocorp05.export',encoding='utf-8') as infile:
    for line in tqdm_notebook(infile):
        #print(line.replace('\t',' ').split(' '))
        cs = line.replace('\t',' ').split(' ')
        ind = ind + 1
        for c in cs:
            if c[0] in a:
                d = c.split('|')
                e = d[2].split('.')
                if len(e) < 3:
                    continue
                if e[2] == 'm':
                    genderList.add((d[0],'m'))
                elif e[2] == 'f':
                    genderList.add((d[0],'f'))
                elif e[2] == 'any':
                    genderList.add((d[0],'any'))
                else:
                    genderList.add((d[0],'none'))


In [197]:
genderList = list(genderList)

In [199]:
genderList.sort()

In [200]:
genderListCleared = []
for item in genderList:
    exit = False
    for ch in item[0]:
        if ch not in a:
            exit = True
    if exit:
        continue
    genderListCleared.append(item)

Sort conflicts in the hindimonocorp by giving none higher priority and then any

In [201]:
for ind in range(0, len(genderListCleared)-1):
    if genderListCleared[ind][0] == genderListCleared[ind+1][0]:
        if genderListCleared[ind][0]  == 'none' or genderListCleared[ind][1]  == 'none' :
            genderListCleared[ind] = genderListCleared[ind][0], 'none'
            genderListCleared[ind+1] = genderListCleared[ind][0], 'none'
        else:
            genderListCleared[ind] = genderListCleared[ind][0], 'any'
            genderListCleared[ind+1] = genderListCleared[ind][0], 'any'


In [202]:
len(genderListCleared),len(set(genderListCleared))

(290872, 284250)

Remove duplicates

In [203]:
genderListCleared = list(set(genderListCleared))

In [206]:
mCount = 0
fCount = 0
nCount = 0
for item in genderListCleared:
    if item[1] == 'm':
        mCount+=1
    elif item[1] == 'f':
        fCount+=1
    elif item[1] == 'none':
        nCount+=1

In [207]:
mCount,fCount,nCount,len(genderListCleared)-mCount-fCount-nCount

(17803, 9695, 243852, 12900)

Store data of Hindimonocorp

In [232]:
with open('genderListCleared', 'wb') as fp:
    pickle.dump(genderListCleared, fp)


In [209]:
genderListNoNone= []
for item in genderListCleared:
    if item[1] == 'm':
        genderListNoNone.append(item)
    elif item[1] == 'f':
        genderListNoNone.append(item)
    elif item[1] == 'any':
        genderListNoNone.append(item)


Remove none objects and store

In [233]:
with open('genderListNoNone', 'wb') as fp:
    pickle.dump(genderListNoNone, fp)


In [10]:
noneWords = list(set(genderListCleared)-set(genderListNoNone))

In [29]:
noneWords = set([x[0] for x in noneWords])

Import data from hindi dependency tree. Mark objects that have more than one gender tag as any

In [5]:
import lingatagger.genderlist as gndrlist
import lingatagger.tokenizer as tok
from lingatagger.tagger import numericTagger

genders2 = gndrlist.drawlist()
genderList2 = []
for i in genders2:
    x = i.split("\t")
    if type(numericTagger(x[0])[0]) != tuple:
        count = 0
        for ch in list(x[0]):
            if ch not in a:
                count+=1
        if count == 0:
            if len(x)>=3:
                genderList2.append((x[0],'any'))
            else:
                genderList2.append((x[0],x[1]))


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [15]:
genderList2.sort()
genderList2Cleared = genderList2
for ind in range(0, len(genderList2Cleared)-1):
    if genderList2Cleared[ind][0] == genderList2Cleared[ind+1][0]:
        genderList2Cleared[ind] = genderList2Cleared[ind][0], 'any'
        genderList2Cleared[ind+1] = genderList2Cleared[ind][0], 'any'

In [16]:
genderList2Cleared = list(set(genderList2Cleared))

In [17]:
mCount2 = 0
fCount2 = 0
for item in genderList2Cleared:
    if item[1] == 'm':
        mCount2+=1
    elif item[1] == 'f':
        fCount2+=1

In [18]:
mCount2,fCount2,len(genderList2Cleared)-mCount2-fCount2

(10636, 3892, 5604)

Save hindi dependency tree corpus after clearing duplicates

In [19]:
with open('genderList2Cleared', 'wb') as fp:
    pickle.dump(genderList2Cleared, fp)


Remove words that are marked as any in hindi dependency tree but marked as none from hindi mono corp

In [30]:
genderList2Matched = []
for item in genderList2Cleared:
    if item[0] in noneWords:
        continue
    genderList2Matched.append(item)

In [8]:
len(genderList2Cleared)-len(genderList2Matched)

10

Save this new list

In [35]:
with open('genderList2Matched', 'wb') as fp:
    pickle.dump(genderList2Matched, fp)


Merge the two sets and create a new corpus with 60:20:20 split for train:validation:test

In [13]:
mergedList = []
for item in genderList2Matched:
    mergedList.append((item[0], item[1]))
for item in genderListNoNone:
    mergedList.append((item[0], item[1]))
mergedList.sort()

Delete conflicts

In [14]:
mergedListCopy = []

for ind in range(1, len(mergedList)-1):
    if mergedList[ind][0] == mergedList[ind+1][0]:
        continue
    if mergedList[ind-1][0] == mergedList[ind][0]:
        continue
    mergedListCopy.append(mergedList[ind])

mergedList = mergedListCopy

In [12]:
with open('mergedList', 'wb') as fp:
    pickle.dump(mergedList, fp)


Define encoding function and create train and test sets

In [10]:
import numpy as np

def encodex(text):
    s = list(text)
    
    indices = []
    for i in s:
        indices.append(a.index(i))
    encoded = np.zeros([18, len(a)+1], dtype="int")
    #print(len(a)+1)
    k = 0
    for i in indices:
        encoded[k][i] = 1
        k = k + 1
    for i in range(18-len(list(s))):
        encoded[k+i][len(a)] = 1
    return encoded

def encodey(text):
    if text == "f":
        return [1,0,0]
    elif text == "m":
        return [0,0,1]

    else:
        return [0,1,0] 

def genderdecode(genderTag):
    """
    one-hot decoding for the gender tag predicted by the classfier
    Dimension = 2.
    """
    genderTag = list(genderTag[0])
    index = genderTag.index(heapq.nlargest(1, genderTag)[0])
    if index == 0:
        return 'f'
    if index == 2:
        return 'm'
    if index == 1:
        return 'any'


In [11]:

x = []
y = []
for i in mergedList:
    if len(i[0]) > 18:
        continue
    x.append(encodex(i[0]))
    y.append(encodey(i[1]))

    

In [24]:
X_cnn = np.array(x)
Y_cnn = np.array(y)
order = np.random.permutation(X_cnn.shape[0])
np.take(X_cnn,order,axis=0,out=X_cnn)
np.take(Y_cnn,order,axis=0,out=Y_cnn)


array([[1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

Save train and test set for use in LSTM

In [27]:
with open('X_cnn', 'wb') as fp:
    pickle.dump(X_cnn, fp)

with open('Y_cnn', 'wb') as fp:
    pickle.dump(Y_cnn, fp)

In [25]:
len(X_cnn)

40422

In [26]:
X_cnn.shape

(40422, 18, 120)